# Classification Dataset

### Data Mining Project 2024/25

Authors: Nicola Emmolo, Simone Marzeddu, Jacopo Raffi

In [10]:
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.over_sampling import ADASYN
from scipy.stats import ks_2samp

import pandas as pd
import numpy 
from sklearn.model_selection import train_test_split
import seaborn as sb

In [11]:
dataset = pd.read_csv('../../data/complete_dataset_no_outliers.csv', dtype={'cyclist_team': str})
RANDOM_STATE = 42   

In [12]:
dataset['label'] = (dataset['raw_position'] <= 20).astype(int) #1 if top 20, 0 otherwise 

In [13]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565555 entries, 0 to 565554
Data columns (total 38 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   cyclist_url                  565555 non-null  object 
 1   cyclist_name                 565555 non-null  object 
 2   birth_year                   565555 non-null  int64  
 3   weight                       455823 non-null  float64
 4   height                       455823 non-null  float64
 5   nationality                  565555 non-null  object 
 6   race_url                     565555 non-null  object 
 7   race_name                    565555 non-null  object 
 8   points                       565555 non-null  float64
 9   uci_points                   398000 non-null  float64
 10  length                       565555 non-null  float64
 11  climb_total                  451154 non-null  float64
 12  profile                      451154 non-null  float64
 13 

Feature da eliminare:

- cyclist_url (non generalizzabili)
- cyclist_name (non generalizzabili)
- birth_year (gia incluso in age)
- nationality (troppo sbilanciamento nel dataset)
- weight, height (gia incluso in BMI)
- uci_points (troppo pochi e gia inclusi in points)
- race_url (non generalizzabili)
- race_name (non generalizzabili)
- climb_total, profile (gia inclusi in race_physical_effort)
- mostly_tarmac (sbilanciato)
- cyclist_team (non generalizzabili)
- date (non generalizzabili)
- raw_position, position (obv)
- delta (obv) 
- race_year (non generalizzabili)
- race_stage (non generalizzabili)
- cyclist_age_group (??)
- cyclist_climb_power (feature generata con delta, obv)
- points, startlist_quality (gia inclusti in race_prestige)

We must choose between cyclist_age and cyclist_age_group as both these features represent the same information bu give different advantage/disadvantage to our classification process: 
- cyclist_age -> lower dimensionality (no one-hot) but unbalanced representation
- cyclist_age_group -> arbitraty encoding but balanced representation 

we chose to delete cyclist_age as the encoding is arbitraty but it still respects the "order" of the age_groups

In [14]:
columns = ['length', 'race_season', 'cyclist_bmi', 'cyclist_age_group', 'climb_percentage', 'race_physical_effort', 'race_prestige', 'race_year', 
           'previous_mean_position', 'previous_mean_delta', 'previous_mean_cp', 'cyclist_previous_experience', 'label']
dataset = dataset[columns].dropna()

In [15]:
development_dataset = dataset[dataset['race_year'] < 2022]
test_dataset = dataset[dataset['race_year'] >= 2022]

development_dataset = development_dataset.drop(columns=['race_year'])
test_set = test_dataset.drop(columns=['race_year'])
test_label = test_set.pop('label')

In [16]:
label = development_dataset.pop('label')
train_set, val_set, train_label, val_label = train_test_split(development_dataset, label, stratify=label, test_size=0.20, random_state=RANDOM_STATE)

In [17]:
train_label.value_counts(normalize=True)

label
0    0.863164
1    0.136836
Name: proportion, dtype: float64

In [18]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265975 entries, 307074 to 560587
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   length                       265975 non-null  float64
 1   race_season                  265975 non-null  object 
 2   cyclist_bmi                  265975 non-null  float64
 3   cyclist_age_group            265975 non-null  object 
 4   climb_percentage             265975 non-null  float64
 5   race_physical_effort         265975 non-null  float64
 6   race_prestige                265975 non-null  float64
 7   previous_mean_position       265975 non-null  float64
 8   previous_mean_delta          265975 non-null  float64
 9   previous_mean_cp             265975 non-null  float64
 10  cyclist_previous_experience  265975 non-null  float64
dtypes: float64(9), object(2)
memory usage: 24.4+ MB


In [19]:
def discretize_data(dataset, variables):
    df = dataset.copy()
    for variable in variables:
        #get the unique variable's values
        var = sorted(df[variable].unique())
        
        #generate a mapping from the variable's values to the number representation  
        mapping = dict(zip(var, range(0, len(var) + 1)))

        #add a new colum with the number representation of the variable
        df[variable] = df[variable].map(mapping).astype(int)
    return df

In [20]:
models = [
    RandomOverSampler(random_state=RANDOM_STATE),
    SMOTE(random_state=RANDOM_STATE, k_neighbors=10),
    SMOTENC(random_state=RANDOM_STATE, k_neighbors=10, categorical_features=['race_season']),
    ADASYN(random_state=RANDOM_STATE, n_neighbors=10)
]
oversampling_algorithms = [
    "random",
    "smote_interpolation",
    "smote_interpolation_w_categorical",
    "adasyn"
]
oversampled_datasets = list()

for algorithm, model in zip(oversampling_algorithms, models):
    if algorithm in ("smote_interpolation", "adasyn"):
        oversampled_data, oversampled_labels = model.fit_resample(train_set.select_dtypes(include="number"), train_label)
        oversampled_labels = oversampled_labels.to_numpy().reshape(-1, 1)
        oversampled_dataset = pd.DataFrame(numpy.hstack((oversampled_data, oversampled_labels)), columns=list(train_set.select_dtypes(include="number").columns) + ['label'])

    elif algorithm == "smote_interpolation_w_categorical":
        df = discretize_data(train_set, ['cyclist_age_group'])
        oversampled_data, oversampled_labels = model.fit_resample(df, train_label)
        oversampled_labels = oversampled_labels.to_numpy().reshape(-1, 1)
        oversampled_dataset = pd.DataFrame(numpy.hstack((oversampled_data, oversampled_labels)), columns=(list(df.columns) + ['label']))
    
    else:
        oversampled_data, oversampled_labels = model.fit_resample(train_set, train_label)
        oversampled_labels = oversampled_labels.to_numpy().reshape(-1, 1)
        oversampled_dataset = pd.DataFrame(numpy.hstack((oversampled_data, oversampled_labels)), columns=(list(train_set.columns) + ['label']))
    
    oversampled_dataset["algorithm"] = algorithm

    oversampled_datasets.append(oversampled_dataset)

In [21]:
tests_per_algorithm = list()
train_set['label'] = label
for algorithm, oversampled_dataset in zip(oversampling_algorithms, oversampled_datasets):
    print(algorithm)
    columns = oversampled_dataset.columns
    test_results = list()
    for column in columns:
        if column != "algorithm":
            if not (column == 'cyclist_age_group' and algorithm == "smote_interpolation_w_categorical"):
                test_results.append(ks_2samp(train_set[column], oversampled_dataset[column], alternative="two-sided"))
            else:
                df = discretize_data(train_set, ['cyclist_age_group'])
                test_results.append(ks_2samp(df[column], oversampled_dataset[column], alternative="two-sided"))

    test_data = [(
        test.statistic,
        test.pvalue,
        test.statistic_location
        )
        for test in test_results        
    ]
    test_data = pd.DataFrame(test_data, columns=["KS_test", "p_value", "margin"])
    test_data["algorithm"] = algorithm

    tests_per_algorithm.append(test_data)

validation = pd.concat(tests_per_algorithm, axis="rows")
validation.groupby("algorithm").describe()

random
smote_interpolation
smote_interpolation_w_categorical
adasyn


KS_test                                \
                                    count      mean       std       min   
algorithm                                                                 
adasyn                               10.0  0.077696  0.104526  0.006665   
random                               12.0  0.056894  0.102073  0.001456   
smote_interpolation                  10.0  0.080500  0.104528  0.006120   
smote_interpolation_w_categorical    12.0  0.075489  0.095543  0.006290   

                                                                           \
                                        25%       50%       75%       max   
algorithm                                                                   
adasyn                             0.022468  0.049934  0.075687  0.363682   
random                             0.004987  0.015588  0.064430  0.363164   
smote_interpolation                0.024592  0.050176  0.082586  0.363164   
smote_interpolation_w_categorical  0.028729  0.049869  0.077646  0.363164   

                                  p_value                                   \
                                    count          mean           std  min   
algorithm                                                                    
adasyn                               10.0  6.271167e-08  1.983117e-07  0.0   
random                               12.0  1.332571e-01  3.109593e-01  0.0   
smote_interpolation                  10.0  6.609644e-07  2.090153e-06  0.0   
smote_interpolation_w_categorical    12.0  2.707408e-07  9.378736e-07  0.0   

                                                                     \
                                   25%            50%           75%   
algorithm                                                             
adasyn                             0.0  1.621816e-213  6.753861e-58   
random                             0.0   2.605803e-20  1.965288e-03   
smote_interpolation                0.0  2.778581e-203  3.281960e-75   
smote_interpolation_w_categorical  0.0  6.205062e-198  7.201236e-75   

                                                 
                                            max  
algorithm                                        
adasyn                             6.271167e-07  
random                             8.671228e-01  
smote_interpolation                6.609644e-06  
smote_interpolation_w_categorical  3.248889e-06

Random oversampling seems to mantain the original distribution (p-value = 0.13)

In [22]:
train_oversample = oversampled_datasets[0]
train_oversample['label'].value_counts(normalize=True)

label
0    0.5
1    0.5
Name: proportion, dtype: float64

In [24]:
#drop the categorical variables since we don't need them anymore 
cat_columns = ['race_season']
train_oversample_encoded = discretize_data(train_oversample, ['cyclist_age_group'])
train_oversample_encoded = pd.get_dummies(train_oversample_encoded, columns = cat_columns, prefix_sep='%').drop(columns=['algorithm'])
train_oversample_encoded

,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,label,race_season%autumn,race_season%spring,race_season%summer,race_season%winter
0,171.0,21.052632,0,0.015047,0.049578,0.09184,0.629271,400.275862,0.000426,0.039399,0,False,False,False,True
1,30.0,20.875633,2,0.018033,0.005485,0.09152,0.397673,635.725,0.005308,0.238103,0,False,False,False,True
2,193.0,21.138856,0,0.000166,0.000695,0.058629,0.389288,233.824742,0.0039,0.181055,0,False,False,False,True
3,160.5,21.561909,1,0.012461,0.03617,0.272064,0.457234,319.171875,0.001548,0.095375,0,False,False,True,False
4,219.5,20.715694,1,0.009112,0.098935,0.265181,0.484871,443.283465,0.002125,0.187489,0,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459155,244.0,21.366869,0,0.01025,0.137528,0.11985,0.427095,387.594118,0.009653,0.294718,1,False,True,False,False
459156,229.0,20.006569,2,0.017712,0.313989,0.092961,0.463407,392.43254,0.002853,0.305298,1,True,False,False,False
459157,4.8,24.343809,4,0.002708,0.000006,0.151221,0.471114,456.868098,0.003538,0.481271,1,False,False,True,False
459158,146.6,19.467401,4,0.010368,0.050218,0.137776,0.311876,115.57874,0.003396,0.353673,1,False,False,True,False


In [25]:
val_set['label'] = val_label
val_set = discretize_data(val_set, ['cyclist_age_group'])
cat_columns = ['race_season']
val_set_encoded = pd.get_dummies(val_set, columns = cat_columns, prefix_sep='%')
val_set_encoded

,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,label,race_season%autumn,race_season%spring,race_season%summer,race_season%winter
458045,140.0,21.453287,0,0.014686,0.064868,0.109735,0.303827,215.801887,0.003904,0.149710,0,False,True,False,False
309341,209.0,20.987654,1,0.005450,0.026823,0.294632,0.311496,281.012048,0.001610,0.187828,0,False,True,False,False
255887,200.6,20.307412,4,0.012712,0.115281,0.100131,0.559600,325.286472,0.003063,0.413373,0,False,False,True,False
436558,239.0,23.547881,1,0.013293,0.256682,0.090720,0.257991,379.289720,0.008159,0.558304,0,False,False,False,True
530764,201.6,20.940159,3,0.014008,0.192457,0.324162,0.588222,680.136364,0.001868,0.112961,0,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234418,168.0,22.343516,1,0.023839,0.379089,0.120618,0.548354,502.418605,0.001287,0.248319,0,False,False,True,False
302008,176.0,18.937003,0,0.023364,0.326201,0.102308,0.557270,569.947368,0.000512,0.142862,0,False,True,False,False
37259,172.2,22.530864,4,0.012172,0.122012,0.058228,0.418661,431.135496,0.002757,0.615296,0,False,True,False,False
310072,128.0,19.036421,2,0.018797,0.069405,0.072714,0.562556,463.882716,0.001212,0.254162,0,False,False,False,True


In [26]:
test_set['label'] = test_label
test_set = discretize_data(test_set, ['cyclist_age_group'])
cat_columns = ['race_season']
test_set_encoded = pd.get_dummies(test_set, columns = cat_columns, prefix_sep='%')
test_set_encoded

,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,label,race_season%autumn,race_season%spring,race_season%summer,race_season%winter
195,203.9,21.705129,3,0.009858,0.092363,0.225487,0.729362,470.333333,0.000047,0.009954,0,False,False,False,True
197,204.2,20.756387,0,0.008159,0.076668,0.264741,0.906142,591.500000,0.000036,0.002329,0,False,False,False,True
198,203.9,20.756387,0,0.009858,0.092363,0.225487,0.876822,643.666667,0.000031,0.003379,0,False,False,False,True
199,183.7,20.756387,0,0.008977,0.068267,0.228368,0.828177,559.000000,0.000032,0.006341,0,False,False,False,True
437,123.4,18.827160,3,0.018476,0.158518,0.000000,0.454167,211.500000,0.000211,0.000813,0,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565194,193.0,20.305175,4,0.018705,0.235529,0.078556,0.419099,469.501340,0.002286,0.600163,0,False,False,False,True
565195,154.0,20.305175,4,0.010779,0.028805,0.078556,0.420047,468.336898,0.002282,0.600797,0,False,False,False,True
565196,204.1,20.305175,4,0.009882,0.092776,0.304715,0.420876,467.380319,0.002274,0.746805,0,False,False,False,True
565197,273.4,20.305175,4,0.007992,0.134628,0.399068,0.421826,468.395225,0.002271,0.604920,0,False,True,False,False


Abbiamo trasformato gli attributi categorici in equivalenti numerici per poter essere processati dai modelli di learning: 
- one-hot encoding: è stato utilizzato quando l'attributo categorico non è di tipo ordinale, in questo caso non siamo interessati a preservare ordine o relazioni di vicinanza tra i valori di questo attributo e pertanto vogliamo massimizzare la dissimilarità tra le classi
- discretizzazione: abbiamo usato questa tecnica solo nel caso dell'attributo cyclist_age_group, in quanto queste classi possiedono relazioni di ordine e vicinanza (attributo ordinale)

In [27]:
train_oversample_encoded.to_csv('../../data/ml_datasets/oversampling/train_set.csv', index=False)
val_set_encoded.to_csv('../../data/ml_datasets/oversampling/val_set.csv', index=False)
test_set_encoded.to_csv('../../data/ml_datasets/oversampling/test_set.csv', index=False)